Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [311]:
NAME = "Vaishnavi Vukku"
COLLABORATORS = "Professor Zia"

---

In [312]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [313]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
display(df)

,StudentID,Name,Degree,Exams,Scores
0,1,"Rodriguez, Pamela",graduate,"exam7 (2017), exam9 (2018), exam3 (2018), exam...","61, 38, 85, 70, 44, 43, 68"
1,2,"Jackson, Kristie",undergraduate,"exam2 (2017), exam2 (2017), exam8 (2018), exam...","66, 69, 85, 62, 92, 72, 44, 50, 23"
2,3,"Curtis, George",graduate,"exam7 (2017), exam6 (2017), exam2 (2017), exam...","68, 73, 62, 72"
3,4,"Casey, Christie",undergraduate,"exam10 (2019), exam4 (2019), exam4 (2019), exa...","41, 52, 52, 70"
4,5,"Yoder, Emily",undergraduate,"exam4 (2019), exam4 (2019), exam4 (2019)","52, 45, 61"
...,...,...,...,...,...
95,96,"Dyer, Benjamin",undergraduate,"exam10 (2019), exam4 (2019), exam6 (2017), exa...","47, 53, 64, 57, 71, 79, 77, 56"
96,97,"Delgado, Jason",undergraduate,"exam1 (2016), exam8 (2018)","72, 83"
97,98,"Cortez, Kenneth",undergraduate,"exam3 (2018), exam5 (2020), exam6 (2017)","83, 55, 71"
98,99,"Short, Elizabeth",undergraduate,"exam3 (2018), exam9 (2018), exam3 (2018), exam...","87, 34, 84, 75, 80, 60, 48, 55, 75"


In [335]:
def insert_degrees(conn, values):
    print(values)
    sql = ''' INSERT INTO Degrees(Degree)
              VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

def insert_exams(conn, values):
    sql = ''' INSERT INTO Exams(Exam, Year)
              VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    conn.commit()
    return cur.lastrowid

def insert_students(conn, values):
    print(values)
    sql = ''' INSERT INTO Students(StudentID, First_name, Last_name, Degree)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    conn.commit()
    return cur.lastrowid

def insert_studentexamscores(conn, values):
    sql = ''' INSERT INTO StudentExamScores(StudentID, Exam, Score)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    conn.commit()
    return cur.lastrowid

def normalize_database(non_normalized_db_filename):
#     Normalize 'non_normalized.db'
#     Call the normalized database 'normalized.db'
#     Function Output: No outputs
#     Requirements:
#     Create four tables
#     Degrees table has one column:
#         [Degree] column is the primary key
    
#     Exams table has two columns:
#         [Exam] column is the primary key column
#         [Year] column stores the exam year
    
#     Students table has four columns:
#         [StudentID] primary key column 
#         [First_Name] stores first name
#         [Last_Name] stores last name
#         [Degree] foreign key to Degrees table
        
#     StudentExamScores table has four columns:
#         [PK] primary key column,
#         [StudentID] foreign key to Students table,
#         [Exam] foreign key to Exams table ,
#         [Score] exam score


    # YOUR CODE HERE
    #connecting to DB
    connection = create_connection(non_normalized_db_filename)
    connection_norm = create_connection('normalized.db', True)
    
    #Table creation queries
    create_table_degrees_sql = """ CREATE TABLE [Degrees] (
        [Degree] TEXT  NOT NULL PRIMARY KEY
    ); """

    create_table_exams_sql = """ CREATE TABLE [Exams] (
        [Exam] TEXT  PRIMARY KEY NOT NULL,
        [Year] INTEGER NOT NULL
    );  """
    
    create_table_students_sql = """ CREATE TABLE [Students] (
    [StudentId] INTEGER  PRIMARY KEY NOT NULL,
    [First_Name] TEXT NOT NULL,
    [Last_Name] TEXT NOT  NULL,
    [Degree] TEXT NOT NULL,
    FOREIGN KEY(Degree) REFERENCES Degrees(Degree)
    );  """
    
    create_table_studentexamscores_sql = """ CREATE TABLE [StudentExamScores] (
    [PK] INTEGER  PRIMARY KEY NOT NULL,
    [StudentId] INTEGER NOT NULL,
    [Exam] TEXT NOT NULL,
    [Score] INTEGER NOT NULL,
    FOREIGN KEY(StudentID) REFERENCES Students(StudentID)
    FOREIGN KEY(Exam) REFERENCES Exams(Exam)
    );  """
    
    #Saving data from non-normalised DB to dataframes
    sql_statement_degrees = "select Degree FROM Students;"
    df_degrees = pd.read_sql_query(sql_statement_degrees, connection)
    
    sql_statement_exams = "select Exams FROM Students;"
    df_exams = pd.read_sql_query(sql_statement_exams, connection)
    
    sql_statement_students = "select StudentID, Name, Degree FROM Students;"
    df_students = pd.read_sql_query(sql_statement_students, connection)
    
    sql_statement_studentexamscores = "select StudentID, Exams, Scores FROM Students;"
    df_studentexamscores = pd.read_sql_query(sql_statement_studentexamscores, connection)
    
    #Normalizing the dataframes
    degrees = []
    for row in df_degrees.itertuples():
        if row[1] not in degrees:
             degrees.append((row[1]))
    
    exams = []
    for row in df_exams.itertuples():
        row_value = row[1].split(',')
        for inner_row in row_value:
            name = inner_row.strip().split(' ')[0]
            year = inner_row.strip().split(' ')[1]
            tuple_exam = (name, year[1:-1])
            if tuple_exam not in exams:
                exams.append(tuple_exam)
    
    students = []
    for row in df_students.itertuples():
        student_id = row[1]
        first_name = row[2].strip().split(',')[1]
        last_name = row[2].strip().split(',')[0]
        degree = row[3]
        tuple_students = (student_id, first_name.strip(), last_name.strip(), degree)
        students.append(tuple_students)
    
    studentexamscores = []
    for row in df_studentexamscores.itertuples():
        tmp = list(row[2].strip().split(' '))
        exam_tmp = []
        for i in range(int(len(tmp)/2)):
            if i == 0:
                exam_tmp.append(tmp[i])
            else:
                exam_tmp.append(tmp[2*i])
        score_tmp = list(row[3].split(','))
        studentid_lst = []
        exam_lst = []
        scores_lst = []   
        for i in range(len(exam_tmp)):
            my_tuple = row[1],exam_tmp[i],score_tmp[i]
            studentexamscores.append(my_tuple)
            
    #Creating Tables in normalised db
    with connection_norm:
        create_table(connection_norm, create_table_degrees_sql)
        create_table(connection_norm, create_table_exams_sql)
        create_table(connection_norm, create_table_students_sql)
        create_table(connection_norm, create_table_studentexamscores_sql)
        
    #Insering the data from dataframes to normalised db
    for values in degrees:
        insert_degrees(connection_norm, (values,))
    for values in exams:
        insert_exams(connection_norm, values)
    for values in students:
        insert_students(connection_norm, values)
    for values in studentexamscores:
        insert_studentexamscores(connection_norm, values)
    

In [336]:
normalize_database('non_normalized.db')
conn = create_connection('normalized.db')


('graduate',)
('undergraduate',)
(1, 'Pamela', 'Rodriguez', 'graduate')
(2, 'Kristie', 'Jackson', 'undergraduate')
(3, 'George', 'Curtis', 'graduate')
(4, 'Christie', 'Casey', 'undergraduate')
(5, 'Emily', 'Yoder', 'undergraduate')
(6, 'Andrew', 'Oliver', 'undergraduate')
(7, 'Kevin', 'Oliver', 'undergraduate')
(8, 'Jennifer', 'Brown', 'graduate')
(9, 'Anthony', 'Murillo', 'undergraduate')
(10, 'Brad', 'Yang', 'undergraduate')
(11, 'Christopher', 'Simmons', 'undergraduate')
(12, 'Craig', 'Griffin', 'undergraduate')
(13, 'Kathy', 'Wheeler', 'graduate')
(14, 'Joseph', 'Rojas', 'undergraduate')
(15, 'Paige', 'Trujillo', 'graduate')
(16, 'Carly', 'Walker', 'undergraduate')
(17, 'Robert', 'Burke', 'graduate')
(18, 'Paul', 'Smith', 'undergraduate')
(19, 'Michael', 'Cherry', 'undergraduate')
(20, 'Ana', 'Rodriguez', 'graduate')
(21, 'Tammy', 'Ellis', 'graduate')
(22, 'Thomas', 'Arellano', 'undergraduate')
(23, 'Bianca', 'Hunter', 'graduate')
(24, 'Michael', 'Bates', 'graduate')
(25, 'Stephani

In [316]:
def ex1():
    # Write an SQL statement that SELECTs all rows from the `Exams` table and sort the exams by Year
    # output columns: exam, year
    
    # YOUR CODE HERE
    sql_statement = "select * from exams order by year asc,Exam asc;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [317]:
sql_statement = ex1()
data = pd.read_csv("ex1.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year
0,exam1,2016
1,exam2,2017
2,exam6,2017
3,exam7,2017
4,exam3,2018
5,exam8,2018
6,exam9,2018
7,exam10,2019
8,exam4,2019
9,exam5,2020


In [318]:
def ex2():
    # Write an SQL statement that SELECTs all rows from the `Degrees` table and sort the degrees by name
    # output columns: degree
    
    # YOUR CODE HERE
    sql_statement = "select * from Degrees order by Degree asc;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [319]:
sql_statement = ex2()
data = pd.read_csv("ex2.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree
0,graduate
1,undergraduate


In [320]:
def ex3():
    # Write an SQL statement that counts the numbers of gradate and undergraduate students
    # output columns: degree, count_degree
    
    # YOUR CODE HERE
    sql_statement = "select degree, count(degree) as count_degree from Students group by degree order by count_degree asc;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [321]:
sql_statement = ex3()
data = pd.read_csv("ex3.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree,count_degree
0,graduate,35
1,undergraduate,65


In [322]:
def ex4():
    # Write an SQL statement that calculates the exam averages for exams; sort by average in descending order.
    # output columns: exam, year, average
    # round to two decimal places
    
    
    # YOUR CODE HERE
    sql_statement = "select StudentExamScores.Exam, Exams.Year, round(avg(Score),2) as average from StudentExamScores inner join Exams on StudentExamScores.Exam = Exams.Exam group by StudentExamScores.Exam order by average desc;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [323]:
sql_statement = ex4()
data = pd.read_csv("ex4.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year,average
0,exam3,2018,86.16
1,exam8,2018,80.18
2,exam1,2016,75.09
3,exam7,2017,68.96
4,exam2,2017,64.28
5,exam6,2017,61.80
6,exam4,2019,54.04
7,exam5,2020,45.56
8,exam10,2019,44.68
9,exam9,2018,39.43


In [324]:
def ex5():
    # Write an SQL statement that calculates the exam averages for degrees; sort by average in descending order.
    # output columns: degree, average 
    # round to two decimal places
    
    # YOUR CODE HERE
    sql_statement = "select Students.Degree, round(avg(Score),2) as average from StudentExamScores inner join Students on StudentExamScores.StudentId = Students.StudentId group by Students.Degree order by average desc;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [325]:
sql_statement = ex5()
data = pd.read_csv("ex5.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree,average
0,graduate,62.50
1,undergraduate,61.63


In [345]:
def ex6():
    # Write an SQL statement that calculates the exam averages for students; sort by average in descending order. Show only top 10 students
    # output columns: first_name, last_name, degree, average
    # round to two decimal places
    
    # YOUR CODE HERE
    sql_statement = "select Students.first_name, Students.last_name, Students.Degree, round(avg(Score),2) as average from StudentExamScores inner join Students on StudentExamScores.StudentId = Students.StudentId group by StudentExamScores.StudentId order by average desc, first_name desc limit 10;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [346]:
sql_statement = ex6()
data = pd.read_csv("ex6.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,First_Name,Last_Name,Degree,average
0,John,Washington,graduate,83.00
1,Robert,Andrade,undergraduate,82.00
2,Paul,Smith,undergraduate,79.00
3,Jason,Delgado,undergraduate,77.50
4,Jason,Thompson,graduate,75.00
5,Calvin,Perez,undergraduate,75.00
6,Calvin,Martinez,undergraduate,74.50
7,Billy,Palmer,undergraduate,74.38
8,Matthew,King,undergraduate,73.56
9,Debra,Pratt,undergraduate,72.60
